In [1]:
!pip install numpy-stl

In [2]:
!pip install torch

In [3]:
!pip install numpy trimesh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 694.7/694.7 kB 17.7 MB/s eta 0:00:0000:01


In [4]:
import numpy as np
from stl import mesh

# Define the 8 vertices of the cube
vertices = np.array([\
    [-1, -1, -1],
    [+1, -1, -1],
    [+1, +1, -1],
    [-1, +1, -1],
    [-1, -1, +1],
    [+1, -1, +1],
    [+1, +1, +1],
    [-1, +1, +1]])
# Define the 12 triangles composing the cube
faces = np.array([\
    [0,3,1],
    [1,3,2],
    [0,4,7],
    [0,7,3],
    [4,5,6],
    [4,6,7],
    [5,1,2],
    [5,2,6],
    [2,3,6],
    [3,7,6],
    [0,1,5],
    [0,5,4]])

# Create the mesh
cube = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))
for i, f in enumerate(faces):
    for j in range(3):
        cube.vectors[i][j] = vertices[f[j],:]

# Write the mesh to file "cube.stl"
cube.save('cube.stl')
save_path = '/path/to/your/directory/cube.stl'

# Write the mesh to the specified file path
cube.save(save_path)


In [5]:
import os

# Define the path of the directory you want to create
directory_path = '/kaggle/working/cubedataset'

# Create the directory
os.makedirs(directory_path, exist_ok=True)

print(f'Directory "{directory_path}" created successfully.')


Directory "/kaggle/working/cubedataset" created successfully.


In [7]:
import numpy as np
from stl import mesh
import os

# Function to create a cube mesh of given size
def create_cube_mesh(size):
    # Define the vertices of the cube
    vertices = np.array([
        [-size, -size, -size],
        [+size, -size, -size],
        [+size, +size, -size],
        [-size, +size, -size],
        [-size, -size, +size],
        [+size, -size, +size],
        [+size, +size, +size],
        [-size, +size, +size]
    ])

    # Define the faces of the cube
    faces = np.array([
        [0, 3, 1],
        [1, 3, 2],
        [0, 4, 7],
        [0, 7, 3],
        [4, 5, 6],
        [4, 6, 7],
        [5, 1, 2],
        [5, 2, 6],
        [2, 3, 6],
        [3, 7, 6],
        [0, 1, 5],
        [0, 5, 4]
    ])

    # Create the mesh
    cube = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))
    for i, f in enumerate(faces):
        for j in range(3):
            cube.vectors[i][j] = vertices[f[j], :]

    return cube

# Directory where STL files will be saved
save_directory = '/kaggle/working/cubedataset'

# Create cubes of sizes from 1 to 50 units and save them as STL files
for size in range(1, 51):
    cube_mesh = create_cube_mesh(size)
    filename = f'cube_size_{size}.stl'
    file_path = os.path.join(save_directory, filename)
    cube_mesh.save(file_path)
    print(f'Saved {filename} at {file_path}')

print('All cubes saved successfully.')


Saved cube_size_1.stl at /kaggle/working/cubedataset/cube_size_1.stl
Saved cube_size_2.stl at /kaggle/working/cubedataset/cube_size_2.stl
Saved cube_size_3.stl at /kaggle/working/cubedataset/cube_size_3.stl
Saved cube_size_4.stl at /kaggle/working/cubedataset/cube_size_4.stl
Saved cube_size_5.stl at /kaggle/working/cubedataset/cube_size_5.stl
Saved cube_size_6.stl at /kaggle/working/cubedataset/cube_size_6.stl
Saved cube_size_7.stl at /kaggle/working/cubedataset/cube_size_7.stl
Saved cube_size_8.stl at /kaggle/working/cubedataset/cube_size_8.stl
Saved cube_size_9.stl at /kaggle/working/cubedataset/cube_size_9.stl
Saved cube_size_10.stl at /kaggle/working/cubedataset/cube_size_10.stl
Saved cube_size_11.stl at /kaggle/working/cubedataset/cube_size_11.stl
Saved cube_size_12.stl at /kaggle/working/cubedataset/cube_size_12.stl
Saved cube_size_13.stl at /kaggle/working/cubedataset/cube_size_13.stl
Saved cube_size_14.stl at /kaggle/working/cubedataset/cube_size_14.stl
Saved cube_size_15.stl a

In [14]:
import numpy as np
from stl import mesh

expected_vertex_count = 108  # Expected count of vertices

def preprocess_stl(file_path, expected_count):
    # Load the STL file
    your_mesh = mesh.Mesh.from_file(file_path)
    vertices = your_mesh.vectors.reshape(-1, 3)
    
    # Check if resizing or padding is needed
    current_count = vertices.shape[0]
    if current_count != expected_count:
        print(f"Warning: Vertices shape mismatch for file {file_path}. Expected {expected_count}, got {current_count}. Resizing vertices.")
        if current_count < expected_count:
            # Pad with zeros if fewer vertices than expected
            pad_size = expected_count - current_count
            padding = np.zeros((pad_size, 3))
            vertices = np.vstack((vertices, padding))
        else:
            # Truncate if more vertices than expected
            vertices = vertices[:expected_count]
    
    return vertices


In [15]:
import torch
from torch.utils.data import Dataset
import trimesh
import numpy as np

class STL3DDataset(Dataset):
    def __init__(self, file_list, expected_count=13824):
        self.file_list = file_list
        self.expected_count = expected_count

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        file_path = self.file_list[idx]
        mesh = trimesh.load(file_path)

        vertices = mesh.vertices
        current_count = vertices.shape[0]
        if current_count != self.expected_count:
            print(f"Warning: Vertices shape mismatch for file {file_path}. Expected {self.expected_count}, got {current_count}. Resizing vertices.")
            if current_count < self.expected_count:
                pad_size = self.expected_count - current_count
                padding = np.zeros((pad_size, 3))
                vertices = np.vstack((vertices, padding))
            else:
                vertices = vertices[:self.expected_count]

        vertices = torch.tensor(vertices, dtype=torch.float32)
        vertices = vertices.flatten()

        return vertices


In [12]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from torchsummary import summary
from tqdm import tqdm
import trimesh


# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [11]:
!pip install torchsummary

In [16]:
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 13824*3),
            nn.Tanh()
        )

    def forward(self, z):
        output = self.model(z)
        return output.view(-1, 13824, 3)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(13824*3, 1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, vertices):
        vertices = vertices.view(vertices.size(0), -1)
        output = self.model(vertices)
        return output


In [ ]:
def get_file_list(directory):
    file_list = []
    for filename in os.listdir(directory):
        if filename.endswith('.stl'):
            file_list.append(os.path.join(directory, filename))
    return file_list

data_directory = "/kaggle/working/cubedataset/"
file_list = get_file_list(data_directory)
dataset = STL3DDataset(file_list)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)


In [17]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 13824*3),
            nn.Tanh()
        )

    def forward(self, z):
        output = self.model(z)
        return output.view(-1, 13824, 3)


ValueError: string is not a file: path/to/stl1.stl

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(13824*3, 1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, vertices):
        vertices = vertices.view(vertices.size(0), -1)
        output = self.model(vertices)
        return output


In [ ]:
generator = Generator().to(device)
discriminator = Discriminator().to(device)
summary(generator, (100,))
summary(discriminator, (13824*3,))


In [ ]:
criterion = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=0.0002)
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002)


In [ ]:
num_epochs = 50
for epoch in range(num_epochs):
    for i, real_vertices in enumerate(dataloader):
        real_vertices = real_vertices.to(device)
        batch_size = real_vertices.size(0)

        # Create labels for real and fake data
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # Train Discriminator
        optimizer_D.zero_grad()
        outputs = discriminator(real_vertices)
        d_loss_real = criterion(outputs, real_labels)
        d_loss_real.backward()

        z = torch.randn(batch_size, 100).to(device)
        fake_vertices = generator(z)
        outputs = discriminator(fake_vertices.detach())
        d_loss_fake = criterion(outputs, fake_labels)
        d_loss_fake.backward()
        optimizer_D.step()

        d_loss = d_loss_real + d_loss_fake

        # Train Generator
        optimizer_G.zero_grad()
        z = torch.randn(batch_size, 100).to(device)
        fake_vertices = generator(z)
        outputs = discriminator(fake_vertices)
        g_loss = criterion(outputs, real_labels)
        g_loss.backward()
        optimizer_G.step()

        if (i + 1) % 1 == 0:
            print(f'Epoch [{epoch}/{num_epochs}], Batch [{i+1}/{len(dataloader)}], D_loss: {d_loss.item():.4f}, G_loss: {g_loss.item():.4f}')


In [ ]:
from pymeshlab import MeshSet

def remesh_stl(file_path, target_vertex_count):
    ms = MeshSet()
    ms.load_new_mesh(file_path)
    ms.meshing_isotropic_explicit_remeshing(targetlen=1.0, exactnumvert=target_vertex_count)
    ms.save_current_mesh(file_path)


In [ ]:
class CubeDataset(Dataset):
    def __init__(self, directory, cube_dim=24):
        self.file_list = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.stl')]
        self.cube_dim = cube_dim

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        filename = self.file_list[idx]
        cube_mesh = mesh.Mesh.from_file(filename)
        vertices = cube_mesh.vectors.reshape(-1)

        expected_size = self.cube_dim ** 3 * 3
        if vertices.shape[0] != expected_size:
            print(f"Warning: Vertices shape mismatch for file {filename}. Expected {expected_size}, got {vertices.shape[0]}. Resizing vertices.")
            if vertices.shape[0] < expected_size:
                vertices = np.pad(vertices, (0, expected_size - vertices.shape[0]), mode='constant', constant_values=0)
            else:
                vertices = vertices[:expected_size]

        return vertices.astype(np.float32)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

class Generator(nn.Module):
    def __init__(self, latent_dim, output_dim):
        super(Generator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, output_dim),
            nn.Tanh()
        )

    def forward(self, z):
        return self.fc(z)

class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.fc(x)

class CubeGAN:
    def __init__(self, directory, latent_dim=100, cube_dim=24, batch_size=64, lr=0.0002):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.latent_dim = latent_dim
        self.cube_dim = cube_dim

        self.generator = Generator(latent_dim, cube_dim**3 * 3).to(self.device)
        self.discriminator = Discriminator(cube_dim**3 * 3).to(self.device)

        self.optimizer_G = optim.Adam(self.generator.parameters(), lr=lr, betas=(0.5, 0.999))
        self.optimizer_D = optim.Adam(self.discriminator.parameters(), lr=lr, betas=(0.5, 0.999))

        self.criterion = nn.BCELoss()

        self.dataset = CubeDataset(directory, cube_dim=cube_dim)
        self.dataloader = DataLoader(self.dataset, batch_size=batch_size, shuffle=True, num_workers=2)

    def train(self, num_epochs):
        for epoch in range(num_epochs):
            for i, real_cubes in enumerate(self.dataloader):
                real_cubes = real_cubes.to(self.device)

                valid = torch.ones(real_cubes.size(0), 1).to(self.device)
                fake = torch.zeros(real_cubes.size(0), 1).to(self.device)

                # Train Discriminator
                self.optimizer_D.zero_grad()

                real_loss = self.criterion(self.discriminator(real_cubes), valid)

                z = torch.randn(real_cubes.size(0), self.latent_dim).to(self.device)
                fake_cubes = self.generator(z)

                fake_loss = self.criterion(self.discriminator(fake_cubes.detach()), fake)

                d_loss = real_loss + fake_loss
                d_loss.backward()
                self.optimizer_D.step()

                # Train Generator
                self.optimizer_G.zero_grad()

                g_loss = self.criterion(self.discriminator(fake_cubes), valid)
                g_loss.backward()
                self.optimizer_G.step()

                if i % 10 == 0:
                    print(f'Epoch [{epoch}/{num_epochs}], Batch [{i}/{len(self.dataloader)}], '
                          f'D_loss: {d_loss.item():.4f}, G_loss: {g_loss.item():.4f}')

    def generate_cube(self, cube_size):
        with torch.no_grad():
            self.generator.eval()
            z = torch.randn(1, self.latent_dim).to(self.device)
            fake_cube = self.generator(z).cpu().numpy().reshape(-1, 3) * cube_size
            return fake_cube

def get_user_input():
    cube_size = int(input("Enter the dimension of the cube you want to generate: "))
    return cube_size

if __name__ == '__main__':
    stl_directory = '/kaggle/working/cubedataset/'
    cube_gan = CubeGAN(stl_directory)
    cube_gan.train(num_epochs=50)

    cube_size = get_user_input()
    generated_cube = cube_gan.generate_cube(cube_size)
    print(f'Generated Cube with dimension {cube_size}:')
    print(generated_cube)
